In [1]:
import pandas as pd
import numpy as np
import random as rd
import seaborn as sbs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import data
import tensorflow as tf
from matplotlib import pyplot
import time
import csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [2]:
file_data = 'DS_Agua_2017_2022_por_ponto.csv'
path_name='data/'
path_name_results='results/'
file_result = 'Result_SVR-LSTM_Model_Hour.csv'

In [3]:
dataset = pd.read_csv(f'{path_name}{file_data}', sep =';', encoding = 'latin1', decimal='.')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 368849 entries, 1 to 368849
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   DT_MEDICAO_HORA         368849 non-null  object 
 1   PRECIPITACAO            368849 non-null  float64
 2   PRESSAO_ATMOSFERICA     368849 non-null  float64
 3   TEMPERATURA_DO_AR_C     368849 non-null  float64
 4   UMIDADE_RELATIVA_DO_AR  368849 non-null  float64
 5   VELOCIDADE_VENTO        368849 non-null  float64
 6   SK_PONTO                368849 non-null  int64  
 7   VL_MEDICAO              368849 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 25.3+ MB


In [5]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

DT_MEDICAO_HORA           0
PRECIPITACAO              0
PRESSAO_ATMOSFERICA       0
TEMPERATURA_DO_AR_C       0
UMIDADE_RELATIVA_DO_AR    0
VELOCIDADE_VENTO          0
SK_PONTO                  0
VL_MEDICAO                0
dtype: int64

In [6]:
def salvar_resultado(sk_ponto, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, Duration):
  #Script to write training cycle results
  data = [sk_ponto, ds_best_param, n_time_steps, MSE, RMSE, MAE, MAPE, Duration]
  fields = ['SK_PONTO','Best Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(data)  
  print(fields)
  print(data)
    
#Script to create the results file
def criar_arquivo_resultado():
  fields = ['SK_PONTO','Best Params','N_Past_Vl','MSE','RMSE','MAE','MAPE','Duration']
  with open(f'{path_name_results}{file_result}', "w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)   

In [7]:
def imprime_grafico(y_test, predict):
  #O gráfico abaixo mostra os valores reais em preto e os valores prditos em azul
  npredicts=1000
  pyplot.figure(figsize=(10,7)) 
  pyplot.scatter(np.arange(0,y_test.shape[0],1)[0:npredicts],np.array(y_test)[0:npredicts], color = 'black')
  pyplot.plot(np.arange(0,predict.shape[0],1)[0:npredicts], np.array(predict)[0:npredicts], color = 'blue', linewidth=2)
  pyplot.xlabel('n seq.')
  pyplot.ylabel('consumo l/h')
  pyplot.show()

In [8]:
def previsao_SVR(sk_ponto, dataset, n_time_steps):

    # dataframe tratament
  df = pd.DataFrame()
  df['VL_MEDICAO']=dataset['VL_MEDICAO']  

  # time serire transform - shit 1 step time

  for n_step in range(1,n_time_steps+1,1):
    df['vl-'+str(n_step)]=dataset['VL_MEDICAO'].shift(n_step)  
    
  df['tp-1']=dataset['TEMPERATURA_DO_AR_C']
  df['pr-1']=dataset['PRESSAO_ATMOSFERICA']
  df['vv-1']=dataset['VELOCIDADE_VENTO']
  df['ur-1']=dataset['UMIDADE_RELATIVA_DO_AR']
  df['ch-1']=dataset['PRECIPITACAO']  

  df.dropna(inplace=True)
  
  
  
  #Split dataset in treinam /  75% treinam  25% test
  nlinhas = int(np.round(df.shape[0] *0.75)) # 
  
  max_size_train_split = int(np.round(nlinhas / 5)) 
  max_size__test_split = int(np.round((df.shape[0] - nlinhas) / 5))
  size_split = 5
  X_train = df.iloc[0:nlinhas,1: 6 + n_time_steps]
  y_train = df.iloc[0:nlinhas,0].values

  X_test = df.iloc[nlinhas:dataset.shape[0],1: 6 + n_time_steps] 
  y_test = df.iloc[nlinhas:dataset.shape[0],0].values
  
  
  # Stores the training execution start time
  Hora_Inicio = time.time()
  
  #Create SVR Regressor
  modelo  = SVR(kernel='rbf')
  
  # Validados cruzado para séries temporais para 5 conjuntos de dados
  ts_cv = TimeSeriesSplit(
      n_splits=size_split, # Number of divisions
      max_train_size=max_size_train_split,    # maximum size of each set. of training
      gap=2, # number of samples to exclude between each training and testing set
      test_size=max_size__test_split, # maximum size of each set. of test.
  )
  C = [ 12550, 125550, 1255555] # Parâmetro de regularização
  gamma = [ 0.00001, 0.000001, 0.0000001, 0.00000001] # Coeficiente da função kernel  
  epsilon = [0.1, 0.01, 0.001, 0.0001]

  hyper_params = [{'kernel': ['rbf'],'C': C, 'gamma':gamma, 'epsilon':epsilon}]  

  modelo = SVR(max_iter=500)
  grid = GridSearchCV(modelo,param_grid=hyper_params,verbose=20,n_jobs=-4,cv=ts_cv, scoring='neg_mean_absolute_percentage_error')
  grid.fit(np.array(X_train), np.array(y_train))
  
  resultado = str(grid.best_params_)
  

  predict=grid.predict(np.array(X_test))   
  
  #Estimate Et for treinam and test
  # For treinam data
  Lt_train = grid.predict(X_train)
  Yt_train = y_train
  Et_train = Yt_train - Lt_train  

  # For test data
  Lt_test = predict
  Yt_test = y_test
  Et_test = Yt_test - Lt_test  

  #create  dataframe of Et value
  df_Et = pd.DataFrame(Et_train, columns=['VL_Et'])  

  for n_step in range(1,n_time_steps+1,1):
      df_Et['vl-'+str(n_step)]=df_Et['VL_Et'].shift(n_step)    

  #drop rows with null values
  df_Et.dropna(inplace=True)  

  Nt_train = df_Et['VL_Et']  

  df_Et.drop(['VL_Et'],axis=1, inplace=True)  

  X_train = df_Et
  y_train = Nt_train
  
  #LSTM Model for hybrid prediction for non linear component Et_train 
  
  train_feature = np.array(X_train).reshape((X_train.shape[0], 1, X_train.shape[1])) # dados de entrada de treinamento.
  train_label = np.array(y_train).reshape((y_train.shape[0], 1, 1)) # dado de saída esperados no treinamento.  

  N_Nos_Layer1=100  #32 #Número de neurônios do layer 1   # 64
  N_Nos_Layer2=96   #64 # Número de neurônios do layer 2  # 128
  N_Nos_Layer3=90   #72 # Númer ode neurônios do layer 3  
  
  nlinhas = X_train.shape[0]
  batch_size = 4096
  if(batch_size > nlinhas):
    batch_size = nlinhas
    
  train_data = data.Dataset.from_tensor_slices((train_feature, train_label))
  train_data = train_data.repeat().batch(batch_size, drop_remainder=True)
  EPOCAS = 200  

  model = keras.Sequential()
  model.add(layers.LSTM(N_Nos_Layer1, batch_input_shape=(batch_size, train_feature.shape[1], train_feature.shape[2]),  return_sequences=True, activation='relu'))  
  model.add(layers.LSTM(N_Nos_Layer2, batch_input_shape=(N_Nos_Layer1, train_feature.shape[1], train_feature.shape[2]), return_sequences=True, activation='relu'))  
  model.add(layers.Dense(N_Nos_Layer3, activation="relu"))
  model.add(layers.Dense(1))

  model.compile(loss='mean_absolute_percentage_error', optimizer='adam', metrics=['mape'])  

  resultado = 'SVR-LSTM' + resultado +'-' + 'LSTM('+ str(N_Nos_Layer1) +','+str(N_Nos_Layer2)+','+str(N_Nos_Layer3)+')'

  #history = model.fit(train_feature, train_label, epochs=EPOCAS,shuffle=True,batch_size=batch_size, verbose=2)
  steps_per_epoch = len(train_feature) // batch_size 
  history = model.fit(train_data, epochs=EPOCAS, steps_per_epoch = steps_per_epoch) 
  
  #--------------------------------------------
  
  #create  dataframe of Et value
  df_Et = pd.DataFrame(Et_test, columns=['VL_Et'])
  
  for n_step in range(1,n_time_steps+1,1):
      df_Et['vl-'+str(n_step)]=df_Et['VL_Et'].shift(n_step)  
  
  #drop rows with null values
  df_Et.dropna(inplace=True)
  
  Nt_test = df_Et['VL_Et']
  
  df_Et.drop(['VL_Et'],axis=1, inplace=True)
  
  X_test = df_Et
  y_test = Nt_test
  
  ## predict test values for LSTM Model
  
  val_feature = np.array(X_test).reshape((X_test.shape[0], 1, X_test.shape[1]))
  val_label = np.array(y_test).reshape((y_test.shape[0], 1, 1))
  
  if(batch_size > X_test.shape[0] ):
    batch_size = X_test.shape[0]
  batch_size=1
  test_data = data.Dataset.from_tensor_slices((val_feature, val_label))
  test_data = test_data.repeat().batch(batch_size, drop_remainder=True)
  
  predict = model.predict(test_data, steps=1)
  #predict = model.predict(train_data, steps=1)
  
  val_label=val_label[0:len(predict)]
  
  if(val_label.shape[0] < predict.shape[0]):
    predict=predict[0:val_label.shape[0]]
  else:
    val_label=val_label[0:predict.shape[0]]
  
  if(batch_size > predict.shape[0]):
    batch_size = predict.shape[0]
    
  predict = np.array(predict).reshape(batch_size,1)
  y_test = np.array(val_label).reshape(batch_size,1)
  
  # Recomposition of Nt + Lt form test data
  Lt_test2 = Lt_test[n_time_steps:len(Lt_test)]
  Yt_test2 = Yt_test[n_time_steps:len(Yt_test)]
  Yt_test_predicted = predict + Lt_test2
  MAPE = ((np.mean(np.abs(Yt_test2 - Yt_test_predicted) / (Yt_test2)))) * 100
  
  print('MAPE',MAPE)
  
  predict = Yt_test_predicted[0]
  y_test = Yt_test2
  print('predict=',predict.shape) 
  print('y_test=',y_test.shape) 
  
  # Stores the training execution end time
  Hora_Fim = time.time()   

  #Calculate the duration of the training execution
  Duracao = Hora_Fim - Hora_Inicio   

  #Mean Squared Error (Mean Squared Difference Between Estimated Values and Actual Values) - MSE
  MSE = mean_squared_error(y_test, predict)   

  # Square Root of Mean Error - RMSE
  RMSE = np.sqrt(mean_squared_error(y_test, predict))   

  # Mean Absolute Distance or Mean Absolute Error - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = y_test)   

  #Calculate the MAPE (Mean Absolute Percentage Error)
  MAPE = ((np.mean(np.abs(y_test -predict) / (y_test)))) * 100    

  #O gráfico abaixo mostra os valores reais em preto e os valores prditos em azul
  #imprime_grafico(Yt_train2, Yt_train_predicted)

  salvar_resultado(sk_ponto, resultado, n_time_steps, MSE, RMSE, MAE, MAPE, Duracao)

In [9]:
#selects the identification points of each reservoir
df_unique_pontos = dataset['SK_PONTO'].unique()

#create file to results
criar_arquivo_resultado()

for sk in np.array(df_unique_pontos):
    print('forecast for sk_ponto = ',sk)
    df_ponto = dataset[dataset['SK_PONTO']==sk]
    df_ponto = df_ponto.drop('SK_PONTO',axis=1)
    for n_time_steps in range(1,7): #predict with 1 to 6 past values of medition
        previsao_SVR(sk, df_ponto, n_time_steps)

forecast for sk_ponto =  5
Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Epoch 1/200
7/7 [==============================] - 2s 58ms/step - loss: 100.4713 - mape: 100.4713
Epoch 2/200
7/7 [==============================] - 0s 60ms/step - loss: 100.5536 - mape: 100.5536
Epoch 3/200
7/7 [==============================] - 0s 65ms/step - loss: 100.2148 - mape: 100.2148
Epoch 4/200
7/7 [==============================] - 0s 65ms/step - loss: 100.0016 - mape: 100.0016
Epoch 5/200
7/7 [==============================] - 0s 57ms/step - loss: 100.2026 - mape: 100.2026
Epoch 6/200
7/7 [==============================] - 0s 63ms/step - loss: 100.0293 - mape: 100.0293
Epoch 7/200
7/7 [==============================] - 0s 71ms/step - loss: 100.0901 - mape: 100.0901
Epoch 8/200
7/7 [==============================] - 0s 64ms/step - loss: 100.0745 - mape: 100.0745
Epoch 9/200
7/7 [==============================] - 0s 62ms/step - loss: 99.9617 - mape: 99.9617
Epoch 10/200
7/7 [==============================] - 0s 63ms/step - loss: 100.0215 - mape: 100.0215
Epoch 11/200
7/7 [===

c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Epoch 1/200
7/7 [==============================] - 3s 64ms/step - loss: 101.0864 - mape: 101.0864
Epoch 2/200
7/7 [==============================] - 0s 68ms/step - loss: 101.9896 - mape: 101.9896
Epoch 3/200
7/7 [==============================] - 0s 68ms/step - loss: 101.0428 - mape: 101.0428
Epoch 4/200
7/7 [==============================] - 0s 67ms/step - loss: 99.8819 - mape: 99.8819
Epoch 5/200
7/7 [==============================] - 0s 67ms/step - loss: 100.1094 - mape: 100.1094
Epoch 6/200
7/7 [==============================] - 1s 81ms/step - loss: 100.2335 - mape: 100.2335
Epoch 7/200
7/7 [==============================] - 1s 77ms/step - loss: 99.9925 - mape: 99.9925
Epoch 8/200
7/7 [==============================] - 1s 77ms/step - loss: 99.9784 - mape: 99.9784
Epoch 9/200
7/7 [==============================] - 1s 96ms/step - loss: 99.7794 - mape: 99.7794
Epoch 10/200
7/7 [==============================] - 1s 85ms/step - loss: 99.8267 - mape: 99.8267
Epoch 11/200
7/7 [===========

...

Epoch 1/200
3/3 [==============================] - 2s 70ms/step - loss: 99.0714 - mape: 99.0714
Epoch 2/200
3/3 [==============================] - 0s 81ms/step - loss: 97.6005 - mape: 97.6005
Epoch 3/200
3/3 [==============================] - 0s 87ms/step - loss: 94.8800 - mape: 94.8800
Epoch 4/200
3/3 [==============================] - 0s 83ms/step - loss: 91.2011 - mape: 91.2011
Epoch 5/200
3/3 [==============================] - 0s 88ms/step - loss: 87.5625 - mape: 87.5625
Epoch 6/200
3/3 [==============================] - 0s 76ms/step - loss: 81.0621 - mape: 81.0621
Epoch 7/200
3/3 [==============================] - 0s 75ms/step - loss: 74.2177 - mape: 74.2177
Epoch 8/200
3/3 [==============================] - 0s 77ms/step - loss: 69.5679 - mape: 69.5679
Epoch 9/200
3/3 [==============================] - 0s 76ms/step - loss: 66.1395 - mape: 66.1395
Epoch 10/200
3/3 [==============================] - 0s 75ms/step - loss: 51.4115 - mape: 51.4115
Epoch 11/200
3/3 [=====================

c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Epoch 1/200
3/3 [==============================] - 1s 64ms/step - loss: 99.6975 - mape: 99.6975
Epoch 2/200
3/3 [==============================] - 0s 80ms/step - loss: 98.9100 - mape: 98.9100
Epoch 3/200
3/3 [==============================] - 0s 76ms/step - loss: 97.6132 - mape: 97.6132
Epoch 4/200
3/3 [==============================] - 0s 71ms/step - loss: 95.8563 - mape: 95.8563
Epoch 5/200
3/3 [==============================] - 0s 74ms/step - loss: 93.4157 - mape: 93.4157
Epoch 6/200
3/3 [==============================] - 0s 72ms/step - loss: 89.0604 - mape: 89.0604
Epoch 7/200
3/3 [==============================] - 0s 75ms/step - loss: 79.9577 - mape: 79.9577
Epoch 8/200
3/3 [==============================] - 0s 79ms/step - loss: 70.6692 - mape: 70.6692
Epoch 9/200
3/3 [==============================] - 0s 79ms/step - loss: 59.1493 - mape: 59.1493
Epoch 10/200
3/3 [==============================] - 0s 80ms/step - loss: 45.7663 - mape: 45.7663
Epoch 11/200
3/3 [=====================

c:\Python\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


Epoch 1/200
3/3 [==============================] - 2s 64ms/step - loss: 100.8450 - mape: 100.8450
Epoch 2/200
3/3 [==============================] - 0s 71ms/step - loss: 99.3277 - mape: 99.3277
Epoch 3/200
3/3 [==============================] - 0s 66ms/step - loss: 97.9964 - mape: 97.9964
Epoch 4/200
3/3 [==============================] - 0s 73ms/step - loss: 96.2379 - mape: 96.2379
Epoch 5/200
3/3 [==============================] - 0s 74ms/step - loss: 93.7286 - mape: 93.7286
Epoch 6/200
3/3 [==============================] - 0s 81ms/step - loss: 89.1125 - mape: 89.1125
Epoch 7/200
3/3 [==============================] - 0s 84ms/step - loss: 77.1770 - mape: 77.1770
Epoch 8/200
3/3 [==============================] - 0s 82ms/step - loss: 62.7669 - mape: 62.7669
Epoch 9/200
3/3 [==============================] - 0s 76ms/step - loss: 63.8726 - mape: 63.8726
Epoch 10/200
3/3 [==============================] - 0s 73ms/step - loss: 68.1077 - mape: 68.1077
Epoch 11/200
3/3 [===================